In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from time import sleep
import pandas as pd

In [5]:
# Configurações do Edge para rodar em modo headless
options = Options()
options.add_argument("--headless")

# Instalação e configuração do serviço Edge
service = Service(EdgeChromiumDriverManager().install())
driver = webdriver.Edge(options=options, service=service)

def get_data(url):
    # Extrair o ano da URL
    year = url.split("serie-a-")[-1].split("/results")[0]  # Extrai o ano diretamente antes de "/results"

    driver.get(url)
    sleep(10)
    
    # Rolagem para o final da página para garantir que todos os elementos sejam carregados
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(10)  # Espera adicional para garantir que a página tenha carregado completamente após a rolagem

    gameRows = driver.find_elements(By.XPATH, '//div[@class="group flex"]')
    rowData = []

    # Adiciona o ano como primeira coluna
    for row in gameRows:
        row_content = row.text.split("\n")
        rowData.append([year] + row_content)

    return rowData


In [7]:
def get_all_data(years, pages):
    base_url = "https://www.oddsportal.com/football/brazil/serie-a-{year}/results/#/page/{page}/"
    all_data = []

    for year in years:
        for page in range(1, pages + 1):
            url = base_url.format(year=year, page=page)
            data = get_data(url)
            all_data.extend(data)

    return all_data

# Anos a serem coletados
years = [2021, 2022, 2023]
# Número de páginas por ano
pages = 8

# Coleta os dados
collected_data = get_all_data(years, pages)


In [8]:
# Criar um DataFrame a partir da lista
df = pd.DataFrame(collected_data, columns=['Year', 'Time', 'Home Team', 'Home Score', 'vs', 'Away Score', 'Away Team', 'Odd 1', 'Odd X', 'Odd 2', 'Bookmakers'])

# Limpar colunas desnecessárias
df.drop(columns=['vs', 'Bookmakers'], inplace=True)

# Salvar o DataFrame em um arquivo Excel
df.to_excel('brasileirao_odds.xlsx', index=False)

print(df)

      Year   Time    Home Team Home Score Away Score       Away Team Odd 1  \
0     2021  21:30   America MG          2          0       Sao Paulo  2.00   
1     2021  21:30  Atletico GO          2          0     Flamengo RJ  1.72   
2     2021  21:30   Bragantino          1          0   Internacional  1.90   
3     2021  21:30    Juventude          1          0     Corinthians  2.25   
4     2021  21:30   Fluminense          3          0  Chapecoense-SC  1.19   
...    ...    ...          ...        ...        ...             ...   ...   
1155  2023  18:30  Atletico-MG          2          1           Goias  1.57   
1156  2023  18:30        Bahia          1          1    Athletico-PR  2.30   
1157  2023  18:30       Cuiaba          2          1       Fortaleza  2.77   
1158  2023  18:30       Santos          0          0       Sao Paulo  2.68   
1159  2023  18:30        Vasco          2          1      America MG  1.48   

     Odd X  Odd 2  
0     3.32   3.98  
1     3.89   4.57  
2  

In [11]:
# contando o numero de dados para year = 2021
print(df[df['Year'] == '2023'].count())

Year          400
Time          400
Home Team     400
Home Score    400
Away Score    400
Away Team     400
Odd 1         400
Odd X         400
Odd 2         400
dtype: int64


In [13]:
base_url = "https://www.oddsportal.com/football/brazil/serie-a-{year}/results/#/page/{page}/"

year = 2022
pages = 8

data2022 = []

for page in range(1, pages + 1):
            url = base_url.format(year=year, page=page)
            data = get_data(url)
            data2022.extend(data)


In [15]:
df22 = pd.DataFrame(data2022, columns=['Year', 'Time', 'Home Team', 'Home Score', 'vs', 'Away Score', 'Away Team', 'Odd 1', 'Odd X', 'Odd 2', 'Bookmakers'])

df22.drop(columns=['vs', 'Bookmakers'], inplace=True)

df22.head()

,Year,Time,Home Team,Home Score,Away Score,Away Team,Odd 1,Odd X,Odd 2
0,2022,16:00,America MG,1,1,Atletico GO,1.59,4.30,5.21
1,2022,16:00,Athletico-PR,3,0,Botafogo RJ,2.18,3.27,3.45
2,2022,16:00,Bragantino,0,1,Fluminense,2.44,3.53,2.79
3,2022,16:00,Ceara,4,1,Juventude,1.67,3.89,5.07
4,2022,16:00,Corinthians,0,1,Atletico-MG,3.05,3.11,2.48


In [23]:
# Filtrar para remover as linhas do ano 2022
df_filtered = df[df['Year'] != '2022']

# Concatenar df_filtered com df22 para obter os dados completos
df_final = pd.concat([df_filtered, df22], ignore_index=True)


In [25]:
# contando o numero de dados para year = 2021
print(df_final[df_final['Year'] == '2022'].count())

Year          400
Time          400
Home Team     400
Home Score    400
Away Score    400
Away Team     400
Odd 1         400
Odd X         400
Odd 2         400
dtype: int64


In [26]:
# Salvar o DataFrame em um arquivo Excel
df.to_excel('brasileirao_odds.xlsx', index=False)

# salvar como csv
df.to_csv('brasileirao_odds.csv', index=False)